In [38]:
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [3]:
masechtot=[]
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/שמות מסכתות.txt') as f:
    data = f.read()
    masechtot = data.split(', ')
    
masechtot_bavli =[]
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/שמות מסכתות בבלי.txt') as f:
    data = f.read()
    masechtot_bavli = data.split(', ')

perek_names = []
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/perakim_names1.txt') as f:
    data = f.read()
    perek_names = data.split(', ')
perek_names2= []
for name in perek_names:
    if len(name.split())>1:     
        if name.split()[0] == "רבי":
            perek_names.append("ר' ".join(name.split()[1:]))
    if "\"" in name:
        perek_names2.append(name.replace("\"","\\\""))
for name in perek_names2:
    perek_names.append(name)

perek_names_bavli = []
with open('/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/perakim_names_Bavli.txt') as f:
    data = f.read()
    perek_names_bavli = data.split(', ')
    
pirkei_mishna=[]
with open("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/perakim_names_MishnaOnly.txt",'r') as f:
    data = f.read()
    pirkei_mishna = data.split(', ')
    
numbers = ["קמא","אחרון","בתרא","עשירי","תשיעי","שמיני","שביעי","ששי","חמישי","דביעי","שלישי","שני","ראשון"]
for letter in ["י","ט","ח","ז","ו","ה","ד","ג","ב","א"]:
        numbers.append(letter+"'")
        numbers.append("י\""+letter)
        numbers.append("כ\""+letter)
rashi_dir= '/Users/ephraimmeiri/gitEtc/Rashi Thesis work/Rashi plaintext/json filtered'

rashi_masechtot = ['Arakhin', 'Bekhorot', 'Chullin', 'Keritot', 'Menachot', 'Temurah', 'Zevachim', 'Beitzah', 'Chagigah', 'Eruvin', 'Pesachim', 'Megillah', 'Rosh Hashanah', 'Shabbat', 'Sukkah', 'Yoma', 'Gittin', 'Ketubot', 'Kiddushin', 'Sotah', 'Yevamot', 'Bava Batra', 'Bava Kamma', 'Bava Metzia', 'Avodah Zarah', 'Makkot', 'Sanhedrin', 'Shevuot', 'Niddah', 'Berakhot']

In [28]:
def get_rashi_file(masechet):
    masechet = masechet.replace("%20"," ")
    file = rashi_dir+"/Rashi on "+masechet+".json"
    if os.path.exists(file):
        with open(file) as f:
            data = json.load(f)
            return data
    else:
        return None

def get_results(query):
    results = []
    for file in os.listdir(rashi_dir):
        if file == '.DS_Store.json':
            continue
        name = file.split(' - ')[0][9:-5]
        print(name,end='\t')
        with open(rashi_dir+"/"+file) as f:
            data = json.load(f)
            for daf,d in zip(data['text'],range(len(data['text']))):
                for line,l in zip(daf,range(len(daf))):
                    for comment,c in zip(line,range(len(line))):
                        cont = query(comment)
                        if cont:
                            results.append([name,d,c,cont,comment])
    return results

def get_right_neighbors(query,dist):
    results = []
    for name in rashi_masechtot:
        file = get_rashi_file(name)
        if file:
            for d in range(len(file['text'])):
                for l in range(len(file['text'][d])):
                    for c in range(len(file['text'][d][l])):
                        cont = query(file['text'][d][l][c],name)
                        if cont:
                            for mas in cont.split(', '):
                                strs= file['text'][d][l][c].split(mas) 
                                words = file['text'][d][l][c].split()
                                words1= [strs[0].split(),strs[1].split()]
                                if len(words1[1])>1 and len(words1[0])>1 and (words[len(words)-1] != words1[1][0]) :
                                    if words1[1]:  # Check if words1[1] is not empty
                                        words1[1][0] = words1[0][-1].join(strs[1][0]) if words1[0] else ""  # Check if words1[0] is not empty before accessing its last element
                                    if words1[0]:  # Check if words1[0] is not empty
                                        words1[0] = words1[0][:-1]
                                if len(words1[0])>dist:
                                    results.append("".join(words1[0][-dist:]))
    # map to frequency
    res_set = set(results)
    freq = {}
    for res in res_set:
        freq[res] = results.count(res)
    freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    return  freq

# Perakim 2, only external

In [22]:
new_name_df= pd.read_csv("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/references subproject/input/PerakimByMaschetBavli.csv",names=["English","Masechet","Perakim"],index_col=False)

In [5]:
new_name_df

,English,Masechet,Perakim
0,Berakhot,ברכות,"מאימתי, היה קורא, מי שמתו, תפילת השחר, אין עומ..."
1,Shabbat,שבת,"יציאות השבת, במה מדליקין, כירה, במה טומנין, במ..."
2,Eruvin,עירובין,"מבוי שהוא גבוה, עושין פסין, בכל מערבין, מי שהו..."
3,Pesachim,פסחים,"אור לארבעה עשר, כל שעה, אלו עוברין, ואלו עוברי..."
4,Rosh Hashanah,"ראש השנה, רה","ארבעה ראשי שנים, אם אינן מכירין, ראוהו בית דין..."
5,Yoma,"יומא, סדר יומא","שבעת ימים, בראשונה, אמר להם הממונה, טרף בקלפי,..."
6,Sukkah,סוכה,"סוכה, הישן תחת המטה, לולב הגזול, לולב וערבה, ה..."
7,Beitzah,ביצה,"ביצה, יום טוב, אין צדין, המביא, משילין"
8,Taanit,"תענית, תעניות","מאימתי, סדר תעניות כיצד, סדר תעניות אלו, פירקא..."
9,Megillah,מגילה,"מגילה נקראת, הקורא למפרע, הקורא עומד, בני העיר"


In [6]:
new_name_df[new_name_df["English"]=="Berakhot"]

,English,Masechet,Perakim
0,Berakhot,ברכות,"מאימתי, היה קורא, מי שמתו, תפילת השחר, אין עומ..."


In [8]:
new_name_df[new_name_df["English"]=="Berakhot"]['Perakim'].values[0].split(", ")

['מאימתי',
 'היה קורא',
 'מי שמתו',
 'תפילת השחר',
 'אין עומדין',
 'כיצד מברכין',
 'שלשה שאכלו',
 'אלו דברים',
 'הרואה']

In [23]:
def contains_perek2(text,current_masechet):
    masechet_perakim= new_name_df[new_name_df["English"]==current_masechet]['Perakim'].values[0].split(", ")
    filtered_perakim = [name for name in perek_names if name not in masechet_perakim]
    for name in filtered_perakim:
        if "ב"+name in text or "בפרק "+name in text:
            return name
def get_results2(query):
    results = []
    for file in os.listdir(rashi_dir):
        if file == '.DS_Store.json':
            continue
        name = file.split(' - ')[0][9:-5]
        print(name,end='\t')
        with open(rashi_dir+"/"+file) as f:
            data = json.load(f)
            for daf,d in zip(data['text'],range(len(data['text']))):
                for line,l in zip(daf,range(len(daf))):
                    for comment,c in zip(line,range(len(line))):
                        cont = query(comment,name)
                        if cont:
                            results.append([name,d,c,cont,comment])
    return results
bPerek_results = get_results2(contains_perek2)

Bava Kamma	Sanhedrin	Bava Batra	Avodah Zarah	Makkot	Yevamot	Kiddushin	Niddah	Pesachim	Menachot	Sukkah	Yoma	Sotah	Shevuot	Temurah	Gittin	Bekhorot	Berakhot	Arakhin	Chullin	Rosh Hashanah	Ketubot	Zevachim	Megillah	Beitzah	Shabbat	Eruvin	Bava Metzia	Chagigah	Keritot	

In [24]:
bPerek_results_df = pd.DataFrame(bPerek_results,columns=["Masechet","Daf","Line","Perakim","Comment"])
bPerek_results_df

,Masechet,Daf,Line,Perakim,Comment
0,Bava Kamma,3,0,גופה,נגיפה - שדחפה בגופה והזיקה בכוונה:
1,Bava Kamma,7,1,שואל,שומר חנם - חייב בפשיעה והשואל חייב אפילו באונס...
2,Bava Kamma,8,0,שואל,תחת נתינה כו' - דבכולהו כתיב או תחת וגמרינן תח...
3,Bava Kamma,9,0,גדיש,לפטור בו את הטמון - שאם היו בגדים טמונים בגדיש...
4,Bava Kamma,12,0,הנזקין,ובעל חוב דינו בבינונית - במס' גיטין בפרק הנזקי...
...,...,...,...,...,...
2577,Keritot,50,0,הריני נזיר,ספק נזיר - נזיר ספק נטמא ספק לא נטמא דאילו ספק...
2578,Keritot,50,1,שני נזירים,לא נייתי - חטאת העוף הבא על הספק דאמרינן במסכת...
2579,Keritot,50,0,כל הבשר,מה להנך שכן אין עשויין להשתנות - ואף על גב דהך...
2580,Keritot,51,0,שק,מתני' שתי סלעים - זהו דין אשם דכתיב בערכך כסף ...


In [26]:
bPerek_sums = bPerek_results_df.groupby("Perakim").size().reset_index(name="Count")
bPerek_sums

,Perakim,Count
0,אבות הטומאות,1
1,אוכלין,26
2,אור לארבעה עשר,1
3,אותו ואת בנו,6
4,אחד דיני ממונות,6
...,...,...
252,תולין,6
253,תינוקת,4
254,תמיד נשחט,10
255,תנור,92


In [29]:
bperek_right_neighbors = get_right_neighbors(contains_perek2,2)

In [30]:
bperek_right_neighbors 

[('-ב', 32),
 ('במסכתשבת', 11),
 ('במסכתנדה', 10),
 ('בבבאקמא', 9),
 ('אלאב', 9),
 ('-דאמר', 8),
 ('כדאמרינןביבמות', 8),
 ('רבייוסי', 8),
 ('דרבייוסי', 8),
 ('בשחיטתחולין', 7),
 ('במסכתגיטין', 7),
 ('במסכתיומא', 6),
 ('דאמר-', 6),
 ('משנההיא', 5),
 ('כדאמרינןבפסחים', 5),
 ('יוסיב', 5),
 ('-ביבמות', 5),
 ('-בב"ק', 5),
 ('במסכתזבחים', 5),
 ('והכיאמרינן', 5),
 ('בבבאמציעא', 5),
 ('-מפרש', 5),
 ('והכימפרש', 4),
 ('פלוגתאהיא', 4),
 ('דבריםהנעשים', 4),
 ('ופלוגתאהיא', 4),
 ('כדאמרינןב', 4),
 ('בבבאבתרא', 4),
 ('-לקמן', 4),
 ('דקיימאלן', 4),
 ('-דקי"ל', 4),
 ('במסכתמנחות', 4),
 ('בשחיטתקדשים', 4),
 ('ממדינהלמדינה', 3),
 ('-כדאמרינן', 3),
 ('-דכתיב', 3),
 ('יליףלה', 3),
 ('להביבמות', 3),
 ('וקיימאלן', 3),
 ("במס'נדה", 3),
 ("במס'יומא", 3),
 ('נפקאלן', 3),
 ("ר'יוסי", 3),
 ("במס'גיטין", 3),
 ('מפרשלה', 3),
 ('נמיאמרינן', 3),
 ('הוהכדאמרינן', 3),
 ('במסכתתמורה', 3),
 ('לרבייוסי', 3),
 ("-כדאמרי'", 3),
 ('לעובדכוכבים', 3),
 ('בכלעת', 3),
 ('היהכדאמרינן', 3),
 ('-במנחות', 3),
 ('קיימאלן', 3),
 ("ל

In [41]:
dist=1
results = []
for name in rashi_masechtot:
    file = get_rashi_file(name)
    if file:
        masechet_perakim= new_name_df[new_name_df["English"]==name]['Perakim'].values[0].split(", ")
        filtered_perakim = [name for name in perek_names if name not in masechet_perakim]
        for d in range(len(file['text'])):
            for l in range(len(file['text'][d])):
                for c in range(len(file['text'][d][l])):
                    # cont = query(file['text'][d][l][c])
                    for perek in filtered_perakim:
                        # if "במסכת "+masechet+" " in file['text'][d][l][c]:
                        cont = contains_perek2(file['text'][d][l][c],name)
                        if cont:
                            strs = file['text'][d][l][c].split()
                            words = file['text'][d][l][c].split()
                            words1 = [strs[0].split(), strs[1].split()]
                            if len(words1[1]) > dist:
                                neighbor = "".join(words1[1][:dist])
                                results.append("".join(words1[1][:dist]))
# map to frequency
res_set = set(results)
freq = {}
for res in res_set:
    freq[res] = results.count(res)
freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)

KeyboardInterrupt: 

In [40]:
results

[]